In [ ]:
import pandas as pd
import numpy as np
import os
import json
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
df = pd.read_csv('/content/drive/My Drive/Data/gtddata.csv')

In [ ]:
for i in df.columns:
  if df[i].dtype == 'object':
    if len(df[i].unique()) > 10000:
      print(i)

city
summary
corp1
target1
scite1


In [ ]:
df[['corp1', 'target1','scite1']].iloc[0]

corp1                                Cairo Police Department
target1                            Cairo Police Headquarters
scite1     "Police Chief Quits," Washington Post, January...
Name: 0, dtype: object

In [ ]:
df.scite1

0        "Police Chief Quits," Washington Post, January...
1        Committee on Government Operations United Stat...
2        Tom Bates, "Rads: The 1970 Bombing of the Army...
3        Committee on Government Operations United Stat...
4        Committee on Government Operations United Stat...
                               ...                        
93515    "With grenades and gunfire they attacked the P...
93516    "Rebels launch deadly attacks on army position...
93517    "Gunmen blow up power pylon in Parwan," Afghan...
93518    "Boko Haram Releases Abducted Catholic Priest ...
93519    "Spring Revolution Daily News for 2-7 July 202...
Name: scite1, Length: 93520, dtype: object

In [ ]:
corpus = df.scite1
for c in corpus:
    urls = re.findall('(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', c)
    if len(urls) == 0:
        pass
    else:
        print(f'Description: {list(corpus).index(c)} - Links: {urls}')

Description: 447 - Links: [('https', 'translate.googleusercontent.com', '/translate_c?anno=2&depth=1&hl=en&rurl=translate.google.com&sl=pl&sp=nmt4&tl=en&u=https://www.rmf24.pl/fakty/polska/news-46-lat-temu-wysadzil-aule-wsp-w-opolu-zmarl-ryszard-kowalczy,nId,2454041&xid=17259,1500003,15700021,15700124,15700149,15700168,15700186,15700191,15700201,15700208&usg=ALkJrhgk2WiK-nF3PkL1d0bNbW13mzTDpg#utm_source=paste&utm_medium=paste&utm_campaign=firefox')]
Description: 490 - Links: [('http', 'news.bbc.co.uk', '/onthisday/hi/dates/stories/september/19/newsid_2523000/2523027.stm')]
Description: 490 - Links: [('http', 'news.bbc.co.uk', '/onthisday/hi/dates/stories/september/19/newsid_2523000/2523027.stm')]
Description: 490 - Links: [('http', 'news.bbc.co.uk', '/onthisday/hi/dates/stories/september/19/newsid_2523000/2523027.stm')]
Description: 490 - Links: [('http', 'news.bbc.co.uk', '/onthisday/hi/dates/stories/september/19/newsid_2523000/2523027.stm')]
Description: 490 - Links: [('http', 'news.

KeyboardInterrupt: ignored

In [ ]:
# Replacing sites and hiperlinks
corpus_wo_hiperlinks = []
for c in corpus:
    c = re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', 'link', c)
    corpus_wo_hiperlinks.append(c)

# Replacing numbers
corpus_wo_numbers = []
for c in corpus_wo_hiperlinks:
    c = re.sub('\d+(?:\.\d*(?:[eE]\d+))?', 'number', c)
    corpus_wo_numbers.append(c)
corpus_wo_numbers[399]

# Replacing special characteres with whitespace
corpus_text = []
for c in corpus_wo_numbers:
    c = re.sub(r'\W', ' ', c)
    corpus_text.append(c)
corpus_text[1113]

cleaned_corpus = pd.Series(corpus_text).apply(lambda x: x.lower())
cleaned_corpus[990]

cleaned_corpus.index = df['scite1'].index

df['scite1'] = cleaned_corpus

df['scite1']

0         police chief quits   washington post  january...
1        committee on government operations united stat...
2        tom bates   rads  the number bombing of the ar...
3        committee on government operations united stat...
4        committee on government operations united stat...
                               ...                        
93515     with grenades and gunfire they attacked the p...
93516     rebels launch deadly attacks on army position...
93517     gunmen blow up power pylon in parwan   afghan...
93518     boko haram releases abducted catholic priest ...
93519     spring revolution daily news for number numbe...
Name: scite1, Length: 93520, dtype: object

In [ ]:
!pip install transformers

import torch
from transformers import AutoTokenizer, AutoModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.3 MB/s eta 0:00:00


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased").to(device)

In [ ]:
dfs = []
for i in range(0, len(df), 100):
  j = i+100
  if j>len(df)-1:
    j = len(df)
  dfs.append(df.iloc[i:j])

In [ ]:
tokenizeds = []
c = 1
max_length = 0

# Assuming dfs is a list of dataframes or similar structure
for i in dfs:
    tokenized = tokenizer(i.scite1.to_list(), padding=True, truncation=True, return_tensors="pt")

    # Update the maximum sequence length if needed
    max_length = max(max_length, tokenized['input_ids'].shape[1])

    tokenizeds.append(tokenized)
    print(c, '/', len(dfs))
    c += 1

1 / 936
2 / 936
3 / 936
4 / 936
5 / 936
6 / 936
7 / 936
8 / 936
9 / 936
10 / 936
11 / 936
12 / 936
13 / 936
14 / 936
15 / 936
16 / 936
17 / 936
18 / 936
19 / 936
20 / 936
21 / 936
22 / 936
23 / 936
24 / 936
25 / 936
26 / 936
27 / 936
28 / 936
29 / 936
30 / 936
31 / 936
32 / 936
33 / 936
34 / 936
35 / 936
36 / 936
37 / 936
38 / 936
39 / 936
40 / 936
41 / 936
42 / 936
43 / 936
44 / 936
45 / 936
46 / 936
47 / 936
48 / 936
49 / 936
50 / 936
51 / 936
52 / 936
53 / 936
54 / 936
55 / 936
56 / 936
57 / 936
58 / 936
59 / 936
60 / 936
61 / 936
62 / 936
63 / 936
64 / 936
65 / 936
66 / 936
67 / 936
68 / 936
69 / 936
70 / 936
71 / 936
72 / 936
73 / 936
74 / 936
75 / 936
76 / 936
77 / 936
78 / 936
79 / 936
80 / 936
81 / 936
82 / 936
83 / 936
84 / 936
85 / 936
86 / 936
87 / 936
88 / 936
89 / 936
90 / 936
91 / 936
92 / 936
93 / 936
94 / 936
95 / 936
96 / 936
97 / 936
98 / 936
99 / 936
100 / 936
101 / 936
102 / 936
103 / 936
104 / 936
105 / 936
106 / 936
107 / 936
108 / 936
109 / 936
110 / 936
111 / 93

In [ ]:
max_length

75

In [ ]:
for tokenized in tokenizeds:
    padded_input_ids = torch.nn.functional.pad(tokenized['input_ids'], pad=(0, max_length - tokenized['input_ids'].shape[1]))
    padded_attention_mask = torch.nn.functional.pad(tokenized['attention_mask'], pad=(0, max_length - tokenized['attention_mask'].shape[1]))

    tokenized['input_ids'] = padded_input_ids
    tokenized['attention_mask'] = padded_attention_mask

    for k, v in tokenized.items():
        tokenized[k] = v.to(device)

In [ ]:
for i in tokenizeds:
  print(i['attention_mask'].device)
  break

cuda:0


In [ ]:
folder_path = '/content/drive/My Drive/Data/Scites/'

file_name = 'scite'

In [ ]:
import gc

In [ ]:
r = 1
n = len(tokenizeds)
for i in tokenizeds:
  with torch.no_grad():
    x = model(input_ids=i['input_ids'], attention_mask=i['attention_mask']).last_hidden_state[:,0,:]
    x = x.to("cpu")
    (pd.DataFrame(x)).to_csv(folder_path + file_name + str(r) + '.csv', index=False)
    del x
    gc.collect()
    print(r, '/', n)
    r += 1

1 / 936
2 / 936
3 / 936
4 / 936
5 / 936
6 / 936
7 / 936
8 / 936
9 / 936
10 / 936
11 / 936
12 / 936
13 / 936
14 / 936
15 / 936
16 / 936
17 / 936
18 / 936
19 / 936
20 / 936
21 / 936
22 / 936
23 / 936
24 / 936
25 / 936
26 / 936
27 / 936
28 / 936
29 / 936
30 / 936
31 / 936
32 / 936
33 / 936
34 / 936
35 / 936
36 / 936
37 / 936
38 / 936
39 / 936
40 / 936
41 / 936
42 / 936
43 / 936
44 / 936
45 / 936
46 / 936
47 / 936
48 / 936
49 / 936
50 / 936
51 / 936
52 / 936
53 / 936
54 / 936
55 / 936
56 / 936
57 / 936
58 / 936
59 / 936
60 / 936
61 / 936
62 / 936
63 / 936
64 / 936
65 / 936
66 / 936
67 / 936
68 / 936
69 / 936
70 / 936
71 / 936
72 / 936
73 / 936
74 / 936
75 / 936
76 / 936
77 / 936
78 / 936
79 / 936
80 / 936
81 / 936
82 / 936
83 / 936
84 / 936
85 / 936
86 / 936
87 / 936
88 / 936
89 / 936
90 / 936
91 / 936
92 / 936
93 / 936
94 / 936
95 / 936
96 / 936
97 / 936
98 / 936
99 / 936
100 / 936
101 / 936
102 / 936
103 / 936
104 / 936
105 / 936
106 / 936
107 / 936
108 / 936
109 / 936
110 / 936
111 / 93

In [ ]:
folder_path = '/content/drive/My Drive/Data/Scites/'

file_name = 'scite1.csv'

df = pd.read_csv(folder_path + file_name)

In [ ]:
folder_path = '/content/drive/My Drive/Data/Scites/'

file_name = 'scite'

for i in range(2, 937):
  sdf = pd.read_csv(folder_path+file_name+str(i)+'.csv')
  df = pd.concat([df, sdf], ignore_index=True)
  print(df.shape)

(200, 768)
(300, 768)
(400, 768)
(500, 768)
(600, 768)
(700, 768)
(800, 768)
(900, 768)
(1000, 768)
(1100, 768)
(1200, 768)
(1300, 768)
(1400, 768)
(1500, 768)
(1600, 768)
(1700, 768)
(1800, 768)
(1900, 768)
(2000, 768)
(2100, 768)
(2200, 768)
(2300, 768)
(2400, 768)
(2500, 768)
(2600, 768)
(2700, 768)
(2800, 768)
(2900, 768)
(3000, 768)
(3100, 768)
(3200, 768)
(3300, 768)
(3400, 768)
(3500, 768)
(3600, 768)
(3700, 768)
(3800, 768)
(3900, 768)
(4000, 768)
(4100, 768)
(4200, 768)
(4300, 768)
(4400, 768)
(4500, 768)
(4600, 768)
(4700, 768)
(4800, 768)
(4900, 768)
(5000, 768)
(5100, 768)
(5200, 768)
(5300, 768)
(5400, 768)
(5500, 768)
(5600, 768)
(5700, 768)
(5800, 768)
(5900, 768)
(6000, 768)
(6100, 768)
(6200, 768)
(6300, 768)
(6400, 768)
(6500, 768)
(6600, 768)
(6700, 768)
(6800, 768)
(6900, 768)
(7000, 768)
(7100, 768)
(7200, 768)
(7300, 768)
(7400, 768)
(7500, 768)
(7600, 768)
(7700, 768)
(7800, 768)
(7900, 768)
(8000, 768)
(8100, 768)
(8200, 768)
(8300, 768)
(8400, 768)
(8500, 768)


In [ ]:
df.shape

(93520, 768)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

normalized_data = scaler.fit_transform(df)

In [ ]:
normalized_data.shape

(93520, 768)

In [ ]:
(pd.DataFrame(normalized_data)).to_csv('/content/drive/My Drive/Data/scitenorm.csv', index=False)

In [ ]:
ndf = pd.read_csv('/content/drive/My Drive/Data/scitenorm.csv')

In [ ]:
ndf.shape

(93520, 768)